In [1]:
import requests
import pandas as pd
from datetime import date, datetime
from pprint import pprint as pp

In [2]:
BASE_URL = "https://api.carbonintensity.org.uk/regional/intensity"
# added base url for sending request to an API.

In [ ]:
last_half_hour = requests.get(BASE_URL).json()["data"][0]
# pp(last_half_hour['from'])
# pp(last_half_hour['intensity'])
print(last_half_hour['regions'][0]['dnoregion'])
pp(last_half_hour)

In [13]:
start = date(2024, 1, 1)
end = date(2024, 1, 3)

int_date_range = requests.get(f"{BASE_URL}/{start}/{end}").json()["data"]
# pp(int_date_range)
int_date_range

[{'from': '2023-12-31T23:30Z',
  'to': '2024-01-01T00:00Z',
  'regions': [{'regionid': 1,
    'dnoregion': 'Scottish Hydro Electric Power Distribution',
    'shortname': 'North Scotland',
    'intensity': {'forecast': 4, 'index': 'very low'},
    'generationmix': [{'fuel': 'biomass', 'perc': 0.6},
     {'fuel': 'coal', 'perc': 0},
     {'fuel': 'imports', 'perc': 0},
     {'fuel': 'gas', 'perc': 0.8},
     {'fuel': 'nuclear', 'perc': 6.8},
     {'fuel': 'other', 'perc': 0},
     {'fuel': 'hydro', 'perc': 44.4},
     {'fuel': 'solar', 'perc': 0},
     {'fuel': 'wind', 'perc': 47.4}]},
   {'regionid': 2,
    'dnoregion': 'SP Distribution',
    'shortname': 'South Scotland',
    'intensity': {'forecast': 24, 'index': 'very low'},
    'generationmix': [{'fuel': 'biomass', 'perc': 3.7},
     {'fuel': 'coal', 'perc': 0},
     {'fuel': 'imports', 'perc': 0.1},
     {'fuel': 'gas', 'perc': 4.8},
     {'fuel': 'nuclear', 'perc': 58.6},
     {'fuel': 'other', 'perc': 0},
     {'fuel': 'hydro', '

In [4]:
extracted_data = []
for entry in int_date_range:
    time = datetime.strptime(entry['from'], '%Y-%m-%dT%H:%MZ')
    date_rec = time.strftime('%Y-%m-%d')
    time_rec = time.strftime('%H:%M')
    day = time.strftime('%A')
    month = time.strftime('%B')
    for region in entry['regions']:
        dnoregion = region['dnoregion']
        regionid = region['regionid']
        intensity_forecast = region['intensity']['forecast']
        intensity_index = region['intensity']['index']
        # generationmix = region['generationmix']
        generation_mix_data = {}
        for fuel_data in region['generationmix']:
            fuel_type = fuel_data['fuel']
            percentage = fuel_data['perc']
            generation_mix_data[fuel_type] = percentage
        extracted_data.append({
            'date': date_rec,
            'from': time_rec,
            'day_recorded': day,
            'month_recorded': month,
            'dnoregion': dnoregion,
            'regionid': regionid,
            'intensity_forecast': intensity_forecast,
            'intensity_index': intensity_index,
            **generation_mix_data
        })

len(extracted_data)
pp(extracted_data)

[{'biomass': 0.6,
  'coal': 0,
  'date': '2023-12-31',
  'day_recorded': 'Sunday',
  'dnoregion': 'Scottish Hydro Electric Power Distribution',
  'from': '23:30',
  'gas': 0.8,
  'hydro': 44.4,
  'imports': 0,
  'intensity_forecast': 4,
  'intensity_index': 'very low',
  'month_recorded': 'December',
  'nuclear': 6.8,
  'other': 0,
  'regionid': 1,
  'solar': 0,
  'wind': 47.4},
 {'biomass': 3.7,
  'coal': 0,
  'date': '2023-12-31',
  'day_recorded': 'Sunday',
  'dnoregion': 'SP Distribution',
  'from': '23:30',
  'gas': 4.8,
  'hydro': 0,
  'imports': 0.1,
  'intensity_forecast': 24,
  'intensity_index': 'very low',
  'month_recorded': 'December',
  'nuclear': 58.6,
  'other': 0,
  'regionid': 2,
  'solar': 0,
  'wind': 32.7},
 {'biomass': 0.9,
  'coal': 0,
  'date': '2023-12-31',
  'day_recorded': 'Sunday',
  'dnoregion': 'Electricity North West',
  'from': '23:30',
  'gas': 0.8,
  'hydro': 0,
  'imports': 0,
  'intensity_forecast': 4,
  'intensity_index': 'very low',
  'month_record

In [ ]:
import psycopg2
import yaml

with open('conn.yaml', 'r') as file:
    config = yaml.safe_load(file)

    host = config.get('host')
    user = config.get('user')
    db = config.get('database')
    password = config.get('password')
    port = config.get('port')

    conn = psycopg2.connect(
    dbname=db,
    user=user,
    password=password,
    host=host,
    port=port
)
    cur = conn.cursor()
    print('succesful!')
    for data_point in extracted_data:
        # Extract values from the data_point dictionary
        date_rec = data_point['date']
        time_rec = data_point['from']
        day_recorded = data_point['day_recorded']
        month_recorded = data_point['month_recorded']
        dnoregion = data_point['dnoregion']
        regionid = data_point['regionid']
        intensity_forecast = data_point['intensity_forecast']
        intensity_index = data_point['intensity_index']
        biomass = data_point.get('biomass')  # Use.get() to handle missing values
        coal = data_point.get('coal')
        imports = data_point.get('imports')
        gas = data_point.get('gas')
        nuclear = data_point.get('nuclear')
        other = data_point.get('other')
        hydro = data_point.get('hydro')
        solar = data_point.get('solar')
        wind = data_point.get('wind')

        # SQL query to insert data
        insert_query = """
            INSERT INTO carbon_intensity ("date", "from", day_recorded, month_recorded, dnoregion, region_id, intensity_forecast, intensity_index, biomass, coal, imports, gas, nuclear, other, hydro, solar, wind)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        # Execute the query with the data
        cur.execute(insert_query, (date_rec, time_rec, day_recorded, month_recorded, dnoregion, regionid, intensity_forecast, intensity_index, biomass, coal, imports, gas, nuclear, other, hydro, solar, wind))

    # Commit the changes to the database
    conn.commit()
    print("Data inserted successfully!")



succesful!
Data inserted successfully!


In [5]:
df = pd.DataFrame(extracted_data)

In [7]:
df.head(20)

,date,from,day_recorded,month_recorded,dnoregion,regionid,intensity_forecast,intensity_index,biomass,coal,imports,gas,nuclear,other,hydro,solar,wind
0,2023-12-31,23:30,Sunday,December,Scottish Hydro Electric Power Distribution,1,4,very low,0.6,0.0,0.0,0.8,6.8,0,44.4,0.0,47.4
1,2023-12-31,23:30,Sunday,December,SP Distribution,2,24,very low,3.7,0.0,0.1,4.8,58.6,0,0.0,0.0,32.7
2,2023-12-31,23:30,Sunday,December,Electricity North West,3,4,very low,0.9,0.0,0.0,0.8,41.3,0,0.0,0.0,57.1
3,2023-12-31,23:30,Sunday,December,NPG North East,4,17,very low,9.4,0.0,2.3,1.4,73.8,0,0.0,0.0,13.1
4,2023-12-31,23:30,Sunday,December,NPG Yorkshire,5,83,low,25.0,0.0,0.0,13.4,0.0,0,0.0,0.0,61.6
5,2023-12-31,23:30,Sunday,December,SP Manweb,6,30,very low,0.3,0.0,0.0,7.4,11.7,0,1.3,0.0,79.2
6,2023-12-31,23:30,Sunday,December,WPD South Wales,7,86,low,0.1,0.0,0.0,21.7,0.9,0,0.0,0.0,77.3
7,2023-12-31,23:30,Sunday,December,WPD West Midlands,8,52,low,2.5,1.9,1.1,7.7,20.8,0,0.4,0.0,65.8
8,2023-12-31,23:30,Sunday,December,WPD East Midlands,9,128,moderate,6.3,6.0,3.0,15.8,4.8,0,0.0,0.0,64.2
9,2023-12-31,23:30,Sunday,December,UKPN East,10,19,very low,0.0,0.0,12.5,2.6,20.5,0,0.0,0.0,64.4


In [10]:
df.shape[0]

1746